In [1]:
import psycopg2
import getpass
import pandas as pd

## mimic database

In [2]:
sqluser = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema_name = 'mimiciii'

# password: feifei
conn = psycopg2.connect(dbname = dbname, user = sqluser, host = host, password = getpass.getpass(prompt='Password: '.format(sqluser)))

Password: ········


In [3]:
cur = conn.cursor()
conn.set_session(autocommit=True)

In [4]:
# Get all table names in this database
query=\
"""
SELECT *
FROM pg_catalog.pg_tables
ORDER BY schemaname, tablename
"""
df = pd.read_sql_query(query, conn)
df.head(10)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,information_schema,sql_features,postgres,None,False,False,False,False
1,information_schema,sql_implementation_info,postgres,None,False,False,False,False
2,information_schema,sql_languages,postgres,None,False,False,False,False
3,information_schema,sql_packages,postgres,None,False,False,False,False
4,information_schema,sql_parts,postgres,None,False,False,False,False
5,information_schema,sql_sizing,postgres,None,False,False,False,False
6,information_schema,sql_sizing_profiles,postgres,None,False,False,False,False
7,mimiciii,admissions,postgres,None,True,False,False,False
8,mimiciii,callout,postgres,None,True,False,False,False
9,mimiciii,caregivers,postgres,None,True,False,False,False


In [5]:
# list schemanames
df.schemaname.value_counts()

pg_catalog            63
mimiciii              43
information_schema     7
Name: schemaname, dtype: int64

## Create a new database

In [6]:
query = 'DROP DATABASE IF EXISTS udacity'
cur.execute(query)

In [7]:
try:
    query="create database udacity"
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [8]:
# close the old connection
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

In [9]:
sqluser = 'postgres'
host = 'localhost'
dbname = 'udacity'

conn = psycopg2.connect(dbname = dbname, user = sqluser, host = host, password = 'feifei')
cur = conn.cursor()
conn.set_session(autocommit=True)

In [10]:
# Get all table names in this database
query=\
"""
SELECT *
FROM pg_catalog.pg_tables
ORDER BY schemaname, tablename
"""
df = pd.read_sql_query(query, conn)
df.head(10)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,information_schema,sql_features,postgres,None,False,False,False,False
1,information_schema,sql_implementation_info,postgres,None,False,False,False,False
2,information_schema,sql_languages,postgres,None,False,False,False,False
3,information_schema,sql_packages,postgres,None,False,False,False,False
4,information_schema,sql_parts,postgres,None,False,False,False,False
5,information_schema,sql_sizing,postgres,None,False,False,False,False
6,information_schema,sql_sizing_profiles,postgres,None,False,False,False,False
7,pg_catalog,pg_aggregate,postgres,None,True,False,False,False
8,pg_catalog,pg_am,postgres,None,True,False,False,False
9,pg_catalog,pg_amop,postgres,None,True,False,False,False


In [11]:
# list schemanames
df.schemaname.value_counts()

pg_catalog            63
information_schema     7
Name: schemaname, dtype: int64

### Create a new table

In [12]:
try:
    query="CREATE TABLE IF NOT EXISTS music_library (album_name varchar, artist_name varchar, year int);"
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [13]:
# Get all table names in this database
query=\
"""
SELECT *
FROM pg_catalog.pg_tables
ORDER BY schemaname, tablename
"""
df = pd.read_sql_query(query, conn)
# list schemanames
df.schemaname.value_counts()

pg_catalog            63
information_schema     7
public                 1
Name: schemaname, dtype: int64

In [14]:

# Get the mimiciii table names in the database, sort by tablename
query=\
"""
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog'
AND schemaname != 'information_schema'
ORDER BY tablename
"""
df = pd.read_sql_query(query, conn)
df

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,music_library,postgres,None,False,False,False,False


In [15]:
try:
    cur.execute("SELECT COUNT(*) from music_library")
except psycopg2.Error as e:
    print(e)

print(cur.fetchall())

[(0,)]


In [16]:
# read the table

query = \
"""
SELECT *
FROM music_library
ORDER by artist_name, year, album_name
LIMIT 5
"""
df = pd.read_sql_query(query, conn)
df

,album_name,artist_name,year


In [17]:
# insert some data

try:
    query="INSERT INTO music_library (album_name, artist_name, year) VALUES (%s, %s, %s)"
    values= ('Let It Be', 'The Beatles', 1970)
    cur.execute(query, values)
except psycopg2.Error as e:
    print(e)


try:
    query="INSERT INTO music_library (album_name, artist_name, year) VALUES (%s, %s, %s)" 
    values = ('Rubber Soul', 'The Beatles', 1965)
    cur.execute(query, values)
except psycopg2.Error as e:
    print(e)    

In [18]:
# read the table

query = \
"""
SELECT *
FROM music_library
ORDER by artist_name, year, album_name
LIMIT 5
"""
df = pd.read_sql_query(query, conn)
df

,album_name,artist_name,year
0,Rubber Soul,The Beatles,1965
1,Let It Be,The Beatles,1970


In [19]:
try:
    cur.execute("SELECT * from music_library")
except psycopg2.Error as e:
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Let It Be', 'The Beatles', 1970)
('Rubber Soul', 'The Beatles', 1965)


In [20]:
try:
    cur.execute("DROP TABLE music_library")
except psycopg2.Error as e:
    print(e)

In [21]:

# Get the mimiciii table names in the database, sort by tablename
query=\
"""
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog'
AND schemaname != 'information_schema'
ORDER BY tablename
"""
df = pd.read_sql_query(query, conn)
df

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity


## Finally close the cursor and connection

In [22]:
cur.close()
conn.close()